## Table 1 : Regression output

https://app.datacamp.com/workspace/w/acce6834-78ac-4e3c-9f36-a37b6da20ffe#output-from-the-logistic--regression-model

In [121]:
import warnings
warnings.filterwarnings("ignore")

In [122]:
import pandas as pd
import numpy as np
df=pd.read_csv("employee_churn_data.csv")
df

,department,promoted,review,projects,salary,tenure,satisfaction,bonus,avg_hrs_month,left
0,operations,0,0.577569,3,low,5.0,0.626759,0,180.866070,no
1,operations,0,0.751900,3,medium,6.0,0.443679,0,182.708149,no
2,support,0,0.722548,3,medium,6.0,0.446823,0,184.416084,no
3,logistics,0,0.675158,4,high,8.0,0.440139,0,188.707545,no
4,sales,0,0.676203,3,high,5.0,0.577607,1,179.821083,no
...,...,...,...,...,...,...,...,...,...,...
9535,operations,0,0.610988,4,medium,8.0,0.543641,0,188.155738,yes
9536,logistics,0,0.746887,3,medium,8.0,0.549048,0,188.176164,yes
9537,operations,0,0.557980,3,low,7.0,0.705425,0,186.531008,yes
9538,IT,0,0.584446,4,medium,8.0,0.607287,1,187.641370,yes


In [123]:


df['salary'] = df['salary'].map({
        'low': 0, 'medium': 1, 'high': 2
    })
df['left'] = df['left'].map({
        "no": 0,"yes": 1,
    })


In [124]:
df["reward"]=((df["promoted"]==0)&(df["bonus"]==0)).replace([True,False], [0,1])

In [125]:
df=df[["department","review","tenure","reward","avg_hrs_month","projects","left"]]

In [126]:
df_dep=df[df["department"]=="admin"].reset_index(drop=True).set_index("department")

In [127]:
df_dep.columns

Index(['review', 'tenure', 'reward', 'avg_hrs_month', 'projects', 'left'], dtype='object')

In [129]:
import statsmodels.formula.api as smf
model = smf.logit("left ~ reward + avg_hrs_month + projects + tenure + review", data = df_dep).fit()

model.summary()

Optimization terminated successfully.
         Current function value: 0.544581
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   left   No. Observations:                  423
Model:                          Logit   Df Residuals:                      417
Method:                           MLE   Df Model:                            5
Date:                Wed, 29 Mar 2023   Pseudo R-squ.:                 0.08351
Time:                        22:21:58   Log-Likelihood:                -230.36
converged:                       True   LL-Null:                       -251.35
Covariance Type:            nonrobust   LLR p-value:                 5.953e-08
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       -41.3129     22.632     -1.825      0.068     -85.671       3.045
reward           -0.1307      0.282     -0.463      0.643      -0.683       0.422
avg_hrs_month     0.2015      0.135      1.492      0.136      -0.063       0.466
projects         -0.0283      0.208     -0.136      0.892      -0.436       0.379
tenure           -0.3674      0.392     -0.938      0.348      -1.135       0.400
review            8.6317      1.511      5.713      0.000       5.670      11.593
=================================================================================
"""

In [130]:
model_odds = pd.DataFrame(np.exp(model.params), columns= ['Odds'])
model_odds['z-value']= model.pvalues
model_odds[['2.5%', '97.5%']] = np.exp(model.conf_int())
model_odds.insert(1,"Log odds",np.log(model_odds["Odds"]))
model_odds = model_odds.rename_axis('Variable').reset_index()
model_odds.insert(0,"Department","admin")
model_odds=round(model_odds,3)
model_odds

,Department,Variable,Odds,Log odds,z-value,2.5%,97.5%
0,admin,Intercept,0.000,-41.313,0.068,0.000,21.011
1,admin,reward,0.878,-0.131,0.643,0.505,1.525
2,admin,avg_hrs_month,1.223,0.202,0.136,0.939,1.594
3,admin,projects,0.972,-0.028,0.892,0.647,1.462
4,admin,tenure,0.692,-0.367,0.348,0.321,1.492
5,admin,review,5606.625,8.632,0.000,290.149,108338.364


https://www.pythonfordatascience.org/logistic-regression-python/

https://towardsdatascience.com/how-to-interpret-the-odds-ratio-with-categorical-variables-in-logistic-regression-5bb38e3fc6a8

In [134]:
0.692/1.692

0.408983451536643

In [136]:
5606.625/5607

0.9999331193151418